<a href="https://colab.research.google.com/github/ankalagigaurave/ML_SEM_V/blob/master/Labs/ML_Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ridge_regression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error

from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
from livelossplot import PlotLossesKeras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../Exercise 9/Absenteeism_at_work_AAA/Absenteeism_at_work.csv', sep = ';')
data.head()


In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
data = data.drop('ID', axis = 1)
cats = ['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Disciplinary failure',
        'Education', 'Son', 'Social drinker', 'Social smoker', 'Pet']

for i in cats:
    data[i] = data[i].astype('category')

In [ ]:
fig = plt.figure(figsize = (17, 10))
sns.heatmap(data.corr(), annot = True, cmap = 'coolwarm')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop('Absenteeism time in hours', axis = 1), 
                                                    data['Absenteeism time in hours'], 
                                                   test_size = 0.3,
                                                   shuffle = True)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
lr = LinearRegression()
parameters = {'fit_intercept':[True, False], 'normalize':[True, False], 'copy_X':[True, False]}

grid = GridSearchCV(lr, parameters, cv = 5)
random = RandomizedSearchCV(lr, parameters, cv = 5)

nn = models.Sequential()

In [ ]:
grid.fit(x_train, y_train)
print('Best Score: ', grid.best_score_)

In [ ]:
random.fit(x_train, y_train)
print('Best Score: ', random.best_score_)

In [ ]:
nn.add(layers.Dense(512, activation = 'relu', input_shape = (19, )))
nn.add(layers.Dense(256, activation = 'relu'))
nn.add(layers.Dense(128))
nn.add(layers.LeakyReLU(alpha = 0.001))
nn.add(layers.Dense(1, activation = 'linear'))

nn.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'mean_squared_error')

In [ ]:
es = EarlyStopping(monitor = 'val_accuracy', patience = 25, verbose = 1, mode = 'min')
callbacks = [es, PlotLossesKeras()]
nn.fit(x_train, y_train, epochs = 100, batch_size = 20, validation_split = 0.3, callbacks = callbacks)